In [ ]:
from importlib import reload
from burst_workflow import bursts
import pystac
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
from pathlib import Path
import fsspec

reload(bursts)

In [ ]:
slc = '../S1A_IW_SLC__1SDV_20211229T231926_20211229T231953_041230_04E66A_3DBE.zip'
df = pd.DataFrame()
burst_df = bursts.update_burst_dataframe(df, slc, swath=1, polarization='vv')

In [ ]:
burst_df

# STAC

In [ ]:
catalog = pystac.Catalog(id='burst-catalog', description='A catalog containing Sentinel-1 burst SLCs')

item = pystac.Item(id='local-image',
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime.utcnow(),
                 properties={})

catalog.add_item(item)

# can I add additional properties to an asset?
item.add_asset(
    key='image',
    asset=pystac.Asset(
        href=img_path,
        media_type=pystac.MediaType.GEOTIFF
    )
)

catalog.normalize_hrefs(os.path.join(tmp_dir.name, 'stac'))

catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

In [ ]:
eo_item = pystac.Item(id='local-image-eo',
                      geometry=footprint,
                      bbox=bbox,
                      datetime=datetime.utcnow(),
                      properties={})

eo_item.ext.enable(pystac.Extensions.EO)
eo_item.ext.eo.apply(bands=wv3_bands)